# Librerías

Importamos las librerías necesarias para procesar la data con la que se va a entrenar el modelo, que en este caso son:
+ numpy
+ keras
+ sklearn

In [1]:
import json
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer

# Descarga del dataset

Se va hacer uso del dataset _Street View House Numbers (SVHN)_ para lo cual podemos descagar tres paquetes de datos.

- train
- test
- extra
---
## Local
Puede descargar y hacer uso de la herramienta `wget` mediante el siguiente comando
```bash
$ pip install wget
```

---
![Google Colab](https://colab.research.google.com/img/colab_favicon_256px.png)
## Desde  Google Colab

Si esta desde este ambiente es importante que monte su cuenta de Drive con el fin de almacenar los archivos `.npy` resultado de este notebook. Modificando `YOUR_PATH` con la carpeta en la que desea trabajar.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/YOUR_PATH'


Si se encuentra desde la plataforma Google Colab puede ejecutar las siguientes celdas para descargar los datasets

In [ ]:
!wget http://ufldl.stanford.edu/housenumbers/train.tar.gz
!wget http://ufldl.stanford.edu/housenumbers/test.tar.gz
!wget http://ufldl.stanford.edu/housenumbers/extra.tar.gz

Los datasets descargados se encuentran comprimidos en formato `tar.gz` por ende procedemos a descomprimirlos con el siguiente comando.

In [ ]:
!tar -xvzf train.tar.gz
!tar -xvzf test.tar.gz
!tar -xvzf extra.tar.gz

# Configuración

Para realizar el preprocesamiento del dataset vamos a hacer uso de una serie de variables de configuración que se van a definir en siguiente celda.

In [9]:
digits_to_predict = 1  				# Cantidad de dígitos que se quieren detectar en cada imagen (DEBE PERMANECER CONSTANTE)

img_size = (180, 180)  				# Tamaño al cual se va a redimensionar las imágenes (DEBE PERMANECER CONSTANTE)
img_channels = 3  					# Número de canales de la imagen (DEBE PERMANECER CONSTANTE)

TRAIN_IMGS_PATH = 'train'  			# Directorio donde se encuentran los datos entrenamiento (MODIFIQUE SI ES NECESARIO)
TEST_IMGS_PATH = 'test'  			# Directorio donde se encuentran los datos prueba (MODIFIQUE SI ES NECESARIO)
EXTRA_TRAIN_IMGS_PATH = 'extra'  	# Directorio donde se encuentran los datos extras (MODIFIQUE SI ES NECESARIO)

# Preprocesamiento de los datasets

Dentro de los recursos del proyecto se podrá en encontrar la carpeta `annotations/` con los archivos `.json` con los etiquetas y características de cada imagen.

Primero procedemos a cargar la información de cada archivo:

In [14]:
# Load the annotations file
with open('annotations/trainDigitStruct.json') as f:
    dataTrain = json.load(f)

with open('annotations/extraDigitStruct.json') as f:
    extraDataTrain = json.load(f)

with open('annotations/testDigitStruct.json') as f:
    dataTest = json.load(f)

Inicializamos los vectores que utilizaremos para almacenar la información correspondiente a las etiquetas, coordenadas de los números y la ubicación de cada imagen.

In [14]:
Xtrain = []
labels = []
bboxes = []
imagePaths = []

Procedemos a definir una función que recibe como parámetro la información que cargamos del `.json` y la ubicación del set de imágenes a procesar, con el fin de dimensionar las coordenadas del cuadro delimitador del dígito en la imagen y agregarlo al los vectores previamente definidos.

In [7]:
def pre_process_images(annotations_file, imgs_folder):
	for item in annotations_file:
		imagePath = os.path.sep.join([imgs_folder, item['filename']])
		image = load_img(imagePath)
		(w, h) = image.size

		if len(item['boxes']) != digits_to_predict:
			continue

		# scale the bounding box coordinates relative to the dimensions of the input image
		image_boxes = []
		img_labels = []
		for i in range(digits_to_predict):
			try:
				startX = item['boxes'][i]['left'] / w
				startY = item['boxes'][i]['top'] / h
				endX = (item['boxes'][i]['left'] + item['boxes'][i]['width']) / w
				endY = (item['boxes'][i]['top'] + item['boxes'][i]['height']) / h
				image_boxes.extend([startX, startY, endX, endY])

				# Fix to change the 10.0 label for a 0
				img_labels.append(0.0 if item['boxes'][i]['label'] == 10.0 else item['boxes'][i]['label'])
			# If the img does not have enough digits in it
			except:
				image_boxes.extend([0, 0, 0, 0])
				img_labels.append(11)

		# load the image and preprocess it
		color_mode = 'rgb' if img_channels == 3 else 'grayscale'
		image = load_img(imagePath, target_size=(img_size), color_mode=color_mode)
		# image = tf.image.rgb_to_grayscale(image)
		image = img_to_array(image) 
		
		Xtrain.append(image)
		labels.append(img_labels)
		bboxes.append(image_boxes)
		imagePaths.append(imagePath)

## Datos de entrenamiento
Procedemos a ejecutar el preprocesamiento de las imágenes correspondientes al set de imágenes de entrenamiento.


In [16]:
pre_process_images(dataTrain, TRAIN_IMGS_PATH)

La siguiente celda añade el dataset extra.

In [ ]:
pre_process_images(extraDataTrain, EXTRA_TRAIN_IMGS_PATH)

### Conversión a NumPy
A continuación se procede a convertir los vectores a vectores de NumPy con el fin hacer operaciones fácilmente.

In [ ]:
# convert the data, class labels, bounding boxes, and image paths to
# NumPy arrays, scaling the input pixel intensities from the range
# [0, 255] to [0, 1]
Xtrain = np.array(Xtrain) / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes)
imagePaths = np.array(imagePaths)

In [19]:
print(Xtrain.shape)
print(labels.shape)
print(bboxes.shape)
print(imagePaths.shape)

(14522, 180, 180, 3)
(14522, 1)
(14522, 4)
(14522,)


### Codificación

Se realiza una codificación One-hot encoding a las etiquetas.

In [20]:

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [21]:
labels.shape

(14522, 10)

### Guardado

Se guardan los datos en formato Numpy para utilizarlos posteriormente.

In [0]:
try:
	os.stat('preprocessed_train/')
except:
	os.mkdir('preprocessed_train/')

np.save(f'preprocessed_train/Xtrain.npy', Xtrain)
np.save(f'preprocessed_train/labels.npy', labels)
np.save(f'preprocessed_train/bboxes.npy', bboxes)
np.save(f'preprocessed_train/imagePaths.npy', imagePaths)

## Datos de prueba

Inicializamos los vectores donde se van a dividir los las etiquetas, coordenadas y ubicación de la imagen.

In [15]:
Xtrain = []
labels = []
bboxes = []
imagePaths = []

In [16]:
pre_process_images(dataTest, TEST_IMGS_PATH)

### Conversión a NumPy
A continuación se procede a convertir los vectores a vectores de NumPy con el fin hacer operaciones fácilmente.

In [17]:

Xtrain = np.array(Xtrain) / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes)
imagePaths = np.array(imagePaths)

### Codificación

Se realiza una codificación One-hot encoding a las etiquetas.

In [18]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [19]:
print(Xtrain.shape)
print(labels.shape)
print(bboxes.shape)
print(imagePaths.shape)

(2483, 180, 180, 3)
(2483, 10)
(2483, 4)
(2483,)


### Guardado

Se guardan los datos en formato Numpy para utilizarlos posteriormente.

In [20]:
try:
	os.stat('preprocessed_test/')
except:
	os.mkdir('preprocessed_test/')

np.save(f'preprocessed_test/Xdata.npy', Xtrain)
np.save(f'preprocessed_test/labels.npy', labels)
np.save(f'preprocessed_test/bboxes.npy', bboxes)
np.save(f'preprocessed_test/imagePaths.npy', imagePaths)

Los archivos guardados en las carpetas preprocessed_train y preprocessed_test con extension `.npy` deben ser almacenados para el siguiente proceso. Si se encuentra en el ambiente Colab se debe ejecutar la siguiente celda para guardar los archivos en la nube dado que por el tamaño de los archivos quedan en el caché de la máquina virtual de google y **NO** persistirán en su Drive.

In [ ]:
# [OPTIONAL] Solo en ambiente Colab
drive.flush_and_unmount();